In [1]:
import os
import pandas as pd
import numpy as npm
import networkx as nx
import csv

df = pd.read_csv('clinic.csv')

df['matrix_functional'] = None
df['matrix_structural'] = None
df['matrix_morphological'] = None

# Creem les noves columnes del dataframe (per tal de poder controlar les metriques de teoria de grafs).
# Functional
df['num_edges_f'] = None
df['mean_degree_f'] = None
df['max_degree_f'] = None
df['mean_density_f'] = None
df['mean_clustering_coefficient_f'] = None
df['assortativity_f'] = None
df['betweenness_centrality_f'] = None
df['global_efficiency_f'] = None
df['local_efficiency_f'] = None
df['transitivity_f'] = None
df['modularity_f'] = None

# Structural
df['num_edges_s'] = None
df['mean_degree_s'] = None
df['max_degree_s'] = None
df['mean_density_s'] = None
df['mean_clustering_coefficient_s'] = None
df['assortativity_s'] = None
df['betweenness_centrality_s'] = None
df['global_efficiency_s'] = None
df['local_efficiency_s'] = None
df['transitivity_s'] = None
df['modularity_s'] = None

# Morphological
df['num_edges_m'] = None
df['mean_degree_m'] = None
df['max_degree_m'] = None
df['mean_density_m'] = None
df['mean_clustering_coefficient_m'] = None
df['assortativity_m'] = None
df['betweenness_centrality_m'] = None
df['global_efficiency_m'] = None
df['local_efficiency_m'] = None
df['transitivity_m'] = None
df['modularity_m'] = None

In [2]:
import numpy as np

type_data_func = '_r_matrix.csv' # Functional data
type_data_str = '_FA_factor.csv' # Structural data
type_data_morp = '_GM_matrix.csv' # Morphological data

folder_type_func = 'subject_networks_rfMRI_v1'
folder_type_str = 'subject_networks_FA_v1'
folder_type_morp = 'subject_networks_GM_v1-no-thr'

patients_MS = list()
controls_HV = list()
patients_MS_dict = dict()
controls_HV_dict = dict()

df['matrix'] = None

# S'iteren totes les matrius.
for index, row in df.iterrows():
    # Llegim la matriu.
    matrix_func = pd.read_csv(
        f'/home/guillem/Desktop/Data_Science/TFM/data/MULTILAYER/{folder_type_func}/{row["id"]}{type_data_func}', 
        header=None
    )
    matrix_str = pd.read_csv(
        f'/home/guillem/Desktop/Data_Science/TFM/data/MULTILAYER/{folder_type_str}/{row["id"]}{type_data_str}', 
        header=None
    )
    matrix_morp = pd.read_csv(
        f'/home/guillem/Desktop/Data_Science/TFM/data/MULTILAYER/{folder_type_morp}/{row["id"]}{type_data_morp}', 
        header=None
    )
    # S'afegeix la columna matriu superior.
    df.at[index, 'matrix_functional'] = np.triu(matrix_func)[np.triu_indices_from(matrix_func, k=1)]
    df.at[index, 'matrix_structural'] = np.triu(matrix_str)[np.triu_indices_from(matrix_str, k=1)]
    df.at[index, 'matrix_morphological'] = np.triu(matrix_morp)[np.triu_indices_from(matrix_morp, k=1)]
    
    #if row['controls_ms'] == 1:
    #    patients_MS.append(matrix)
    #    patients_MS_dict[f"{row['id']}"] = matrix
    #else:
    #    controls_HV.append(matrix)
    #    controls_HV_dict[f"{row['id']}"] = matrix


* Not Oversampling -> KO

In [19]:
'''
# OVERSAMPLING
matrius_sans_functional = list(df[df['controls_ms'] == 0]['matrix_functional'])
matrius_sans_structural = list(df[df['controls_ms'] == 0]['matrix_structural'])
matrius_sans_morphological = list(df[df['controls_ms'] == 0]['matrix_morphological'])

num_matrius_a_generar = 100
generate_rows = list()

# Iterem totes les matrius a generar.
for over_iter in range(num_matrius_a_generar):
    # Creem la nova matriu de 0s.
    new_matrix_funct = [0 for i in range(0, 2850)] #np.zeros((76, 76))
    new_matrix_struct = [0 for i in range(0, 2850)] #np.zeros((76, 76))
    new_matrix_morph = [0 for i in range(0, 2850)] #np.zeros((76, 76))
    
    count = 0
    # Iterar les connexions i omplir amb valors aleatoris de HV.
    for i in range(0, 76):
        for j in range(i+1, 76):
            # Obtenir valors possibles (HV) per la connexió iterada.
            valors_connexio_funct = [matrix[count] for matrix in matrius_sans_functional]
            valors_connexio_struct = [matrix[count] for matrix in matrius_sans_structural]
            valors_connexio_morph = [matrix[count] for matrix in matrius_sans_morphological]

            # Assignar un valor aleatori entre els valors de la connexió.
            new_matrix_funct[count] = np.random.choice(valors_connexio_funct)
            new_matrix_struct[count] = np.random.choice(valors_connexio_struct)
            new_matrix_morph[count] = np.random.choice(valors_connexio_morph)
            
            count += 1   
    
    # Convert to pd.DataFrame.
    #new_matrix_funct = pd.DataFrame(new_matrix_funct)
    #new_matrix_struct = pd.DataFrame(new_matrix_struct)
    #new_matrix_morph = pd.DataFrame(new_matrix_morph)
    
    # Convert to np.array.
    new_matrix_funct = np.array(new_matrix_funct)
    new_matrix_struct = np.array(new_matrix_struct)
    new_matrix_morph = np.array(new_matrix_morph)
    
    # Afegim rows.
    generate_rows.append({'matrix_functional': new_matrix_funct, 'matrix_structural': new_matrix_struct, 'matrix_morphological': new_matrix_morph, 'id': f'oversampling_{over_iter}', 'controls_ms': 0})

# Afegir rows del oversampling al df.
df = df.append(generate_rows, ignore_index=True)
print(f"Length DF: {len(df)}")

'''

Length DF: 265


In [20]:
'''
# Imputem valors d'edat (sobre la mitjana de MS o HV).
mitjana_controls_HV = df.loc[df['controls_ms'] == 0, 'age'].mean()
mitjana_controls_MS = df.loc[df['controls_ms'] == 1, 'age'].mean()

df.loc[df['controls_ms'] == 0, 'age'] = df.loc[df['controls_ms'] == 0, 'age'].fillna(mitjana_controls_HV)
df.loc[df['controls_ms'] == 1, 'age'] = df.loc[df['controls_ms'] == 1, 'age'].fillna(mitjana_controls_MS)
'''

In [21]:
'''
# Imputem valors de sexe.
proportion_sex = df[df['controls_ms'] == 0]['sex'].value_counts(normalize=True)
proportion_sex

np.random.choice(proportion_sex.index, p=proportion_sex.values)
'''

0.0

In [22]:
'''
def assign_sex_value(row):
    if pd.isna(row['sex']):
        return np.random.choice(proportion_sex.index, p=proportion_sex.values)
    else:
        return row['sex']
    
df['sex'] = df.apply(assign_sex_value, axis=1) #df[(df['controls_ms'] == 0) & df['sex'].isna()]['sex']

print(f"Check 'Sex': {any(df['sex'].isna())}")
print(f"Check 'Age': {any(df['age'].isna())}")
'''

* Has de realitzar una correcció per edat i sexe utilitzant una regressió lineal per comptabilitzar la variabilitat deguda a aquests factors.

In [4]:
# CORRECCIÓ DE CONNECTIVITAT PER EDAT I GENERE (corregir l'efecte d'aquestes variables en les matrius).
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.metrics import accuracy_score
import copy

# Regressió lineal on Variables Independents ('age', 'sex') i Variable Dependent ('connexions de la matriu de connectivitat').
X = df[['age', 'sex']]

# Se obtiene una copia de las matrices originales (para corregir los valores).
df['corrected_matrix_functional'] = df['matrix_functional'].copy()
df['corrected_matrix_structural'] = df['matrix_structural'].copy()
df['corrected_matrix_morphological'] = df['matrix_morphological'].copy()

# Se iteran todas las connexions (Functional).
for connection in range(0, df['matrix_functional'][0].shape[0]):
    # Obtenim els valors de la connexió iterada 'connection'.
    connection_values_Y = [df['matrix_functional'][idx][connection] for idx, row in df.iterrows()] 

    # Dividim les dades en Train i Test.
    X_train, X_test, y_train, y_test = train_test_split(X, connection_values_Y, test_size=0.2, random_state=42)

    # Definim el model de Regressió Lineal.
    model = LinearRegression()

    # Entrenem el model.
    model.fit(X_train, y_train)

    # Duem a terme la predicció.
    y_pred = model.predict(X)

    # Calculem els residuals per cada individu.
    residuals_connection_iterated = connection_values_Y - y_pred
    
    # S'iteren tots els residuals, per tal d'afegir els valors a la matriu original.
    for index in range(0, len(residuals_connection_iterated)):
        # S'obté el valor corregit a partir del valor original + residual.
        df['corrected_matrix_functional'][index][connection] = df['corrected_matrix_functional'][index][connection] + residuals_connection_iterated[index]    

# Se iteran todas las connexions (Structural).
for connection in range(0, df['matrix_structural'][0].shape[0]):
    # Obtenim els valors de la connexió iterada 'connection'.
    connection_values_Y = [df['matrix_structural'][idx][connection] for idx, row in df.iterrows()] 

    # Dividim les dades en Train i Test.
    X_train, X_test, y_train, y_test = train_test_split(X, connection_values_Y, test_size=0.2, random_state=42)

    # Definim el model de Regressió Lineal.
    model = LinearRegression()

    # Entrenem el model.
    model.fit(X_train, y_train)

    # Duem a terme la predicció.
    y_pred = model.predict(X)

    # Calculem els residuals per cada individu.
    residuals_connection_iterated = connection_values_Y - y_pred
    
    # S'iteren tots els residuals, per tal d'afegir els valors a la matriu original.
    for index in range(0, len(residuals_connection_iterated)):
        # S'obté el valor corregit a partir del valor original + residual.
        df['corrected_matrix_structural'][index][connection] = df['corrected_matrix_structural'][index][connection] + residuals_connection_iterated[index]    

        
# Se iteran todas las connexions (Functional).
for connection in range(0, df['matrix_morphological'][0].shape[0]):
    # Obtenim els valors de la connexió iterada 'connection'.
    connection_values_Y = [df['matrix_morphological'][idx][connection] for idx, row in df.iterrows()] 

    # Dividim les dades en Train i Test.
    X_train, X_test, y_train, y_test = train_test_split(X, connection_values_Y, test_size=0.2, random_state=42)

    # Definim el model de Regressió Lineal.
    model = LinearRegression()

    # Entrenem el model.
    model.fit(X_train, y_train)

    # Duem a terme la predicció.
    y_pred = model.predict(X)

    # Calculem els residuals per cada individu.
    residuals_connection_iterated = connection_values_Y - y_pred
    
    # S'iteren tots els residuals, per tal d'afegir els valors a la matriu original.
    for index in range(0, len(residuals_connection_iterated)):
        # S'obté el valor corregit a partir del valor original + residual.
        df['corrected_matrix_morphological'][index][connection] = df['corrected_matrix_morphological'][index][connection] + residuals_connection_iterated[index]    


* OverSampling: Increment de les mostres de Voluntaris Sans (HV)

In [40]:
# OVERSAMPLING
matrius_sans_functional = list(df[df['controls_ms'] == 0]['corrected_matrix_functional'])
matrius_sans_structural = list(df[df['controls_ms'] == 0]['corrected_matrix_structural'])
matrius_sans_morphological = list(df[df['controls_ms'] == 0]['corrected_matrix_morphological'])

num_matrius_a_generar = 100
generate_rows = list()

# Iterem totes les matrius a generar.
for over_iter in range(num_matrius_a_generar):
    # Creem la nova matriu de 0s.
    new_matrix_funct = [0 for i in range(0, 2850)] #np.zeros((76, 76))
    new_matrix_struct = [0 for i in range(0, 2850)] #np.zeros((76, 76))
    new_matrix_morph = [0 for i in range(0, 2850)] #np.zeros((76, 76))
    
    count = 0
    # Iterar les connexions i omplir amb valors aleatoris de HV.
    for i in range(0, 76):
        for j in range(i+1, 76):
            # Obtenir valors possibles (HV) per la connexió iterada.
            valors_connexio_funct = [matrix[count] for matrix in matrius_sans_functional]
            valors_connexio_struct = [matrix[count] for matrix in matrius_sans_structural]
            valors_connexio_morph = [matrix[count] for matrix in matrius_sans_morphological]

            # Assignar un valor aleatori entre els valors de la connexió.
            new_matrix_funct[count] = np.random.choice(valors_connexio_funct)
            new_matrix_struct[count] = np.random.choice(valors_connexio_struct)
            new_matrix_morph[count] = np.random.choice(valors_connexio_morph)
            
            count += 1   
    
    # Convert to pd.DataFrame.
    #new_matrix_funct = pd.DataFrame(new_matrix_funct)
    #new_matrix_struct = pd.DataFrame(new_matrix_struct)
    #new_matrix_morph = pd.DataFrame(new_matrix_morph)
    
    # Convert to np.array.
    new_matrix_funct = np.array(new_matrix_funct)
    new_matrix_struct = np.array(new_matrix_struct)
    new_matrix_morph = np.array(new_matrix_morph)
    
    # Afegim rows.
    generate_rows.append({'corrected_matrix_functional': new_matrix_funct, 'corrected_matrix_structural': new_matrix_struct, 'corrected_matrix_morphological': new_matrix_morph, 'id': f'oversampling_{over_iter}', 'controls_ms': 0})

# Afegir rows del oversampling al df.
df = df.append(generate_rows, ignore_index=True)
print(f"Length DF: {len(df)}")


Length DF: 265


* Implementa la imputació de valors nuls per assegurar-te un bon anàlisi davant les dades incompletes (Tractar 0s) 

In [5]:
hv_total = len(df[df['controls_ms'] == 0])
ms_total = len(df[df['controls_ms'] == 1])

percentage = 0.90

count = 0 
for i in range(0, 76):
    for j in range(i+1, 76):
        # Obtenim el valor mitjà per cada connexió (per MS i HV).
        # Obtenim valors de HV i MS, per Matrius de funct, struct i morph.
        hv_values_funct = [matrix[count] for matrix in df[df['controls_ms'] == 0]['corrected_matrix_functional'] if matrix[count] != 0]
        ms_values_funct = [matrix[count] for matrix in df[df['controls_ms'] == 1]['corrected_matrix_functional'] if matrix[count] != 0]
        hv_values_struct = [matrix[count] for matrix in df[df['controls_ms'] == 0]['corrected_matrix_structural'] if matrix[count] != 0]
        ms_values_struct = [matrix[count] for matrix in df[df['controls_ms'] == 1]['corrected_matrix_structural'] if matrix[count] != 0]
        hv_values_morph = [matrix[count] for matrix in df[df['controls_ms'] == 0]['corrected_matrix_morphological'] if matrix[count] != 0]
        ms_values_morph = [matrix[count] for matrix in df[df['controls_ms'] == 1]['corrected_matrix_morphological'] if matrix[count] != 0]
        
        # HV mean.
        hv_mean_funct = np.mean(hv_values_funct)
        hv_mean_struct = np.mean(hv_values_struct)
        hv_mean_morph = np.mean(hv_values_morph)
        
        # MS mean.
        ms_mean_funct = np.mean(ms_values_funct)
        ms_mean_struct = np.mean(ms_values_struct)
        ms_mean_morph = np.mean(ms_values_morph)        
        
        # Transformació dels 0s de HV pel valor mitjà (en cas que la connexió tingui valor en el 60% dels subjectes).
        # HV
        # Functional
        if len(hv_values_funct) > percentage*hv_total:
            for index, matrix in df[df['controls_ms'] == 0]['corrected_matrix_functional'].iteritems():
                if matrix[count] == 0:
                    matrix[count] = hv_mean_funct
        # Structural
        if len(hv_values_struct) > percentage*hv_total:
            for index, matrix in df[df['controls_ms'] == 0]['corrected_matrix_structural'].iteritems():
                if matrix[count] == 0:
                    matrix[count] = hv_mean_struct
        # Morphological
        if len(hv_values_morph) > percentage*hv_total:
            for index, matrix in df[df['controls_ms'] == 0]['corrected_matrix_morphological'].iteritems():
                if matrix[count] == 0:
                    matrix[count] = hv_mean_morph            
        
        # MS
        # Functional
        if len(ms_values_funct) > percentage*ms_total:
            for index, matrix in df[df['controls_ms'] == 1]['corrected_matrix_functional'].iteritems():
                if matrix[count] == 0:
                    matrix[count] = ms_mean_funct
        # Structural
        if len(ms_values_struct) > percentage*hv_total:
            for index, matrix in df[df['controls_ms'] == 1]['corrected_matrix_structural'].iteritems():
                if matrix[count] == 0:
                    matrix[count] = ms_mean_struct
        # Morphological
        if len(ms_values_morph) > percentage*hv_total:
            for index, matrix in df[df['controls_ms'] == 1]['corrected_matrix_morphological'].iteritems():
                if matrix[count] == 0:
                    matrix[count] = ms_mean_morph          
        
        count += 1
        
        

/home/guillem/.local/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/guillem/.local/lib/python3.6/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/guillem/.local/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/guillem/.local/lib/python3.6/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/guillem/.local/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/guillem/.local/lib/python3.6/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/guillem/.local/lib/python3.6/site-packages/numpy/core/f

/home/guillem/.local/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/guillem/.local/lib/python3.6/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/guillem/.local/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/guillem/.local/lib/python3.6/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/guillem/.local/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/guillem/.local/lib/python3.6/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/guillem/.local/lib/python3.6/site-packages/numpy/core/f

/home/guillem/.local/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/guillem/.local/lib/python3.6/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/guillem/.local/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/guillem/.local/lib/python3.6/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/guillem/.local/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/guillem/.local/lib/python3.6/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/guillem/.local/lib/python3.6/site-packages/numpy/core/f

/home/guillem/.local/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/guillem/.local/lib/python3.6/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/guillem/.local/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/guillem/.local/lib/python3.6/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/guillem/.local/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/guillem/.local/lib/python3.6/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/guillem/.local/lib/python3.6/site-packages/numpy/core/f

/home/guillem/.local/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/guillem/.local/lib/python3.6/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/guillem/.local/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/guillem/.local/lib/python3.6/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/guillem/.local/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/guillem/.local/lib/python3.6/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/guillem/.local/lib/python3.6/site-packages/numpy/core/f

/home/guillem/.local/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/guillem/.local/lib/python3.6/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/guillem/.local/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/guillem/.local/lib/python3.6/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/guillem/.local/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/guillem/.local/lib/python3.6/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/guillem/.local/lib/python3.6/site-packages/numpy/core/f

* Generem els llistats amb les matrius corregides

In [8]:
from sklearn.preprocessing import StandardScaler

# Carreguem matrius.
#functional_data = 'subject_networks_rfMRI_v1'
#structural_data = 'subject_networks_FA_v1'
#morphological_data = 'subject_networks_GM_v1-no-thr'

#type_data_funct = '_r_matrix.csv' # Functional data
#type_data_str = '_FA_factor.csv' # Structural data
#type_data_morp = '_GM_matrix.csv'; # Morphological data

#@deprecated
'''
def generate_list_of_matrices(data, type_data):
    llista_matrices = list()
    
    # Iterem tots els pacients.
    for index, row in df.iterrows():
        matrix = pd.read_csv(
            os.path.join(
                os.getcwd(), data, row['id'] + type_data
            ), header=None
        )
        llista_matrices.append(matrix)

    return llista_matrices

'''
def generate_list_of_matrices_from_dataframe(data, column_matrix):
    llista_matrices = list()

    # Iterem tots els pacients.
    #for index, row in data.iterrows():
    #    matrix = data[column_matrix]
    #    llista_matrices.append(matrix)
    dataframe = data[column_matrix]
    
    # Reconstrucció de la matriu.
    for index_patient in range(0, len(dataframe)):
        dataframe_patient = dataframe[index_patient]
        matrix = np.zeros((76, 76))
        count = 0
        
        for i in range(0, 76):
            for j in range(i+1, 76):
                matrix[i][j] = dataframe_patient[count]                    
                count += 1
        
        llista_matrices.append(matrix)
        
    return llista_matrices

llista_matrices_functional = generate_list_of_matrices_from_dataframe(df, 'corrected_matrix_functional')
llista_matrices_structural = generate_list_of_matrices_from_dataframe(df, 'corrected_matrix_structural')
llista_matrices_morphological = generate_list_of_matrices_from_dataframe(df, 'corrected_matrix_morphological')


* Check Nulls in functional, structural and morphological data 

In [9]:
booleans_functional = [np.isnan(llista_matrices_functional[i]).any() for i in range(0, len(llista_matrices_functional))]
booleans_structural = [np.isnan(llista_matrices_structural[i]).any() for i in range(0, len(llista_matrices_structural))]
booleans_morphological = [np.isnan(llista_matrices_morphological[i]).any() for i in range(0, len(llista_matrices_morphological))]

print(f"Null values for Funcitonal data: {any(booleans_functional)}")
print(f"Null values for Structural data: {any(booleans_structural)}")
print(f"Null values for Morphological data: {any(booleans_morphological)}")



Null values for Funcitonal data: False
Null values for Structural data: False
Null values for Morphological data: False


- És necessari normalitzar les dades per assegurar que totes les mesures siguin comparables entre si per a l'anàlisi multicapa.

In [10]:
from sklearn.preprocessing import StandardScaler
'''
# Carreguem matrius.
functional_data = 'subject_networks_rfMRI_v1'
structural_data = 'subject_networks_FA_v1'
morphological_data = 'subject_networks_GM_v1-no-thr'

type_data_funct = '_r_matrix.csv' # Functional data
type_data_str = '_FA_factor.csv' # Structural data
type_data_morp = '_GM_matrix.csv'; # Morphological data


def generate_list_of_matrices(data, type_data):
    llista_matrices = list()
    
    # Iterem tots els pacients.
    for index, row in df.iterrows():
        matrix = pd.read_csv(
            os.path.join(
                os.getcwd(), data, row['id'] + type_data
            ), header=None
        )
        llista_matrices.append(matrix)

    return llista_matrices

llista_matrices_functional = generate_list_of_matrices(functional_data, type_data_funct)
llista_matrices_structural = generate_list_of_matrices(structural_data, type_data_str)
llista_matrices_morphological = generate_list_of_matrices(morphological_data, type_data_morp)

result_matrix_functional = np.concatenate(llista_matrices_functional, axis=1)
result_matrix_structural = np.concatenate(llista_matrices_structural, axis=1)
result_matrix_morphological = np.concatenate(llista_matrices_morphological, axis=1)
'''
# Normalitzem les dades.
scaler = StandardScaler()

def normalize_matrices(llista_matrices):
    normalized_llista_matrices = list()
    for matrix in llista_matrices:
        normalized_matrix = scaler.fit_transform(matrix)
        normalized_llista_matrices.append(normalized_matrix)
    
    return normalized_llista_matrices

normalized_llista_matrices_functional = normalize_matrices(llista_matrices_functional)
normalized_llista_matrices_structural = normalize_matrices(llista_matrices_structural)
normalized_llista_matrices_morphological = normalize_matrices(llista_matrices_morphological)

normalized_result_matrix_functional = np.concatenate(normalized_llista_matrices_functional, axis=1)
normalized_result_matrix_structural = np.concatenate(normalized_llista_matrices_structural, axis=1)
normalized_result_matrix_morphological = np.concatenate(normalized_llista_matrices_morphological, axis=1)


- Harmonitza els pesos amb ComBat (https://github.com/Jfortin1/ComBatHarmonization), ja que hi ha dos tipus d'adquisicions diferents: MSVIS i FIS.

In [11]:
# Harmonitza els pesos amb neuroCombat, ja que hi ha dos tipus d'adquisicions diferents: MSVIS i FIS.
from neuroCombat import neuroCombat

batch_list = [index for index, row in df.iterrows() for _ in range(0, 76)]

data_combat_functional = neuroCombat(
    dat=normalized_result_matrix_functional,
    covars=pd.DataFrame({'batch': batch_list}),
    batch_col = 'batch'
)['data']

data_combat_structural = neuroCombat(
    dat=normalized_result_matrix_structural,
    covars=pd.DataFrame({'batch': batch_list}),
    batch_col = 'batch'
)['data']

data_combat_morphological = neuroCombat(
    dat=normalized_result_matrix_morphological,
    covars=pd.DataFrame({'batch': batch_list}),
    batch_col = 'batch'
)['data']

# Obtenim el nombre de matrius que volem desfer.
nombre_matrius = len(normalized_llista_matrices_functional)

# Obtenim totes les matrius en format original, despres d'aplicar l'harmonització de pesos i normalització.
normalized_harmonized_functional_matrices = np.split(data_combat_functional, nombre_matrius, axis=1)
normalized_harmonized_structural_matrices = np.split(data_combat_structural, nombre_matrius, axis=1)
normalized_harmonized_morphological_matrices = np.split(data_combat_morphological, nombre_matrius, axis=1)


[neuroCombat] Creating design matrix
[neuroCombat] Standardizing data across features
[neuroCombat] Fitting L/S model and finding priors
[neuroCombat] Finding parametric adjustments
[neuroCombat] Final adjustment of data
[neuroCombat] Creating design matrix
[neuroCombat] Standardizing data across features
[neuroCombat] Fitting L/S model and finding priors
[neuroCombat] Finding parametric adjustments
[neuroCombat] Final adjustment of data
[neuroCombat] Creating design matrix
[neuroCombat] Standardizing data across features
[neuroCombat] Fitting L/S model and finding priors
[neuroCombat] Finding parametric adjustments
[neuroCombat] Final adjustment of data


- Considera la possible aplicació de llindars (<0.1) i el filtratge de connexions que no superin més del 60% en la població de subjectes sans.

In [12]:
percentatge_minim = 0.6

def filter_matrices_by_threhsold(normalized_harmonized_matrices):
    # S'obté la matriu comuna bàsica de 0s.
    matriu_comuna = np.zeros_like(normalized_harmonized_matrices[0])

    # Iterem els subjectes sans.
    for index, row in df[df['controls_ms'] == 0].iterrows():
        # Obtenim la matriu de l'individu sa.
        matrix=normalized_harmonized_matrices[index]

        # Si les posicions de la matriu son més gran que 0, es suma el valor de 1 a la matriu comuna.
        matriu_comuna += (matrix > 0).astype(int)

    # S'obtenen les connexions que no presenten un percentatge minim.
    connexions_a_filtrar = matriu_comuna < percentatge_minim*(len(df[df['controls_ms'] == 0]))

    # Les connexions que no passen el threshold de valor minim de connexions en les altres matrius, s'eliminen assignant un 0 a la matriu comuna.
    matriu_comuna[connexions_a_filtrar] = 0
    
    # Iterem totes les dades.
    for index, row in df.iterrows():
        # Obtenim la matriu iterada.
        matrix_iterated = normalized_harmonized_matrices[index]

        # Filtrar valors
        matrix_iterated[matrix_iterated < 0.1] = 0
        
        # Les connexions que no passen el threshold de valor minim de connexions en les altres matrius, s'eliminen assignant un 0 a la matriu comuna.
        matrix_iterated[matriu_comuna == 0] = 0

        # Tornem a colocar la matriu al llistat.
        normalized_harmonized_matrices[index] = matrix_iterated
    
    return normalized_harmonized_matrices


filtered_normalized_harmonized_functional_matrices = filter_matrices_by_threhsold(normalized_harmonized_functional_matrices)
filtered_normalized_harmonized_structural_matrices = filter_matrices_by_threhsold(normalized_harmonized_structural_matrices)
filtered_normalized_harmonized_morphological_matrices = filter_matrices_by_threhsold(normalized_harmonized_morphological_matrices)


* Oversampling: Increment de les mostres de Voluntaris Sans (HV)

In [19]:
# OVERSAMPLING
functional_data = 'Functional'
structural_data = 'Structural'
morphological_data = 'Morphological'

type_data_funct = '_r_matrix.csv' # Functional data
type_data_str = '_FA_factor.csv' # Structural data
type_data_morp = '_GM_matrix.csv'; # Morphological data

# Get data from filtered, normalized and harmonized.
for index, row in df.iterrows():
    matrix_functional = filtered_normalized_harmonized_functional_matrices[index]
    matrix_structural = filtered_normalized_harmonized_structural_matrices[index]
    matrix_morphological = filtered_normalized_harmonized_morphological_matrices[index]

    df.at[index, 'corrected_matrix_functional'] = np.triu(matrix_functional)[np.triu_indices_from(matrix_functional, k=1)]
    df.at[index, 'corrected_matrix_structural'] = np.triu(matrix_structural)[np.triu_indices_from(matrix_structural, k=1)]
    df.at[index, 'corrected_matrix_morphological'] = np.triu(matrix_morphological)[np.triu_indices_from(matrix_morphological, k=1)]
    

matrius_sans_functional = list(df[df['controls_ms'] == 0]['corrected_matrix_functional'])
matrius_sans_structural = list(df[df['controls_ms'] == 0]['corrected_matrix_structural'])
matrius_sans_morphological = list(df[df['controls_ms'] == 0]['corrected_matrix_morphological'])

num_matrius_a_generar = 100
generate_rows = list()

# Iterem totes les matrius a generar.
for over_iter in range(num_matrius_a_generar):
    # Creem la nova matriu de 0s.
    new_matrix_funct = [0 for i in range(0, 2850)] #np.zeros((76, 76))
    new_matrix_struct = [0 for i in range(0, 2850)] #np.zeros((76, 76))
    new_matrix_morph = [0 for i in range(0, 2850)] #np.zeros((76, 76))
    
    count = 0
    # Iterar les connexions i omplir amb valors aleatoris de HV.
    for i in range(0, 76):
        for j in range(i+1, 76):
            # Obtenir valors possibles (HV) per la connexió iterada.
            valors_connexio_funct = [matrix[count] for matrix in matrius_sans_functional]
            valors_connexio_struct = [matrix[count] for matrix in matrius_sans_structural]
            valors_connexio_morph = [matrix[count] for matrix in matrius_sans_morphological]

            # Assignar un valor aleatori entre els valors de la connexió.
            new_matrix_funct[count] = np.random.choice(valors_connexio_funct)
            new_matrix_struct[count] = np.random.choice(valors_connexio_struct)
            new_matrix_morph[count] = np.random.choice(valors_connexio_morph)
            
            count += 1   
    
    # Convert to pd.DataFrame.
    #new_matrix_funct = pd.DataFrame(new_matrix_funct)
    #new_matrix_struct = pd.DataFrame(new_matrix_struct)
    #new_matrix_morph = pd.DataFrame(new_matrix_morph)
    
    # Convert to np.array.
    new_matrix_funct = np.array(new_matrix_funct)
    new_matrix_struct = np.array(new_matrix_struct)
    new_matrix_morph = np.array(new_matrix_morph)
    
    # Afegim rows.
    generate_rows.append({'corrected_matrix_functional': new_matrix_funct, 'corrected_matrix_structural': new_matrix_struct, 'corrected_matrix_morphological': new_matrix_morph, 'id': f'oversampling_{over_iter}', 'controls_ms': 0})

# Afegir rows del oversampling al df.
df = df.append(generate_rows, ignore_index=True)
print(f"Length DF: {len(df)}")


Length DF: 265


* Export Data

In [44]:
functional_data = 'Functional'
structural_data = 'Structural'
morphological_data = 'Morphological'

type_data_funct = '_r_matrix.csv' # Functional data
type_data_str = '_FA_factor.csv' # Structural data
type_data_morp = '_GM_matrix.csv'; # Morphological data

# Export data
for index, row in df.iterrows():
    matrix_funct = np.zeros((76, 76))
    matrix_struct = np.zeros((76, 76))
    matrix_morph = np.zeros((76, 76))

    matrix_functional = row['corrected_matrix_functional']
    matrix_structural = row['corrected_matrix_structural']
    matrix_morphological = row['corrected_matrix_morphological']
    
    count = 0
    for i in range(0, 76):
        for j in range(i+1, 76):
            matrix_funct[i][j] = matrix_functional[count]
            matrix_struct[i][j] = matrix_structural[count]
            matrix_morph[i][j] = matrix_morphological[count]
            
            count += 1
    
    # Output Name
    output_path_f = os.path.join(
        os.getcwd(), functional_data, row['id'] + type_data_funct 
    )
    output_path_s = os.path.join(
        os.getcwd(), structural_data, row['id'] + type_data_str
    )
    output_path_m = os.path.join(
        os.getcwd(), morphological_data, row['id'] + type_data_morp
    )
    
    # Save matrix to CSV.
    with open(output_path_f, mode='w', newline='') as file:
        writer = csv.writer(file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        writer.writerows(matrix_funct)
    with open(output_path_s, mode='w', newline='') as file:
        writer = csv.writer(file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        writer.writerows(matrix_struct)
    with open(output_path_m, mode='w', newline='') as file:
        writer = csv.writer(file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        writer.writerows(matrix_morph)


In [45]:
df.to_csv('clinic_data_oversampled.csv')


In [102]:
from networkx.algorithms import community as nxcom

def get_graph_metrics(graph):
    result = dict()
    
    # 1. Number of nodes.
    num_nodes = graph.number_of_nodes()
    result['num_nodes'] = num_nodes
    
    # 2. Number of edges.
    num_edges = graph.number_of_edges()
    result['num_edges'] = num_edges
    
    # 3. Grau mitjà.
    mean_degree = sum(dict(graph.degree()).values()) / num_nodes
    result['mean_degree'] = mean_degree
    
    # 4. Maximum Degree.
    max_degree = max(dict(graph.degree()).values())
    result['max_degree'] = max_degree
    
    # 5. Density.
    density = nx.density(graph)
    mean_density = np.mean(density)
    result['mean_density'] = mean_density
    
    # 6. Clustering Coefficient.
    clust_coef = nx.average_clustering(graph)
    mean_clust_coef = np.mean(clust_coef)
    result['mean_clust_coef'] = mean_clust_coef
    
    # 7. Assortativity.
    assortativity = nx.degree_assortativity_coefficient(graph)
    result['assortativity'] = assortativity
    
    # 8. Bwetweenness centrality.
    betw_centr = nx.betweenness_centrality(graph)
    result['betw_centr'] = betw_centr
    
    # 9. Global Efficiency.
    Eglobal = nx.global_efficiency(graph)
    result['Eglobal'] = Eglobal
    
    # 10. Local Efficiency.
    Elocal = nx.local_efficiency(graph)
    result['Elocal'] = Elocal
    
    # 11. Transitivity.
    transitivity = nx.transitivity(graph)
    result['transitivity'] = transitivity
    
    # 12. Modularity.
    # Detectem les comunitats a través de la funció girvan_newman.
    comunitats_per_nivell = nxcom.girvan_newman(graph)
    
    groups = list()
    for com in next(comunitats_per_nivell):
        groups.append(list(com))
        
    modularity = nx.community.modularity(graph, groups)
    result['modularity'] = modularity
    
    return result


In [103]:
# Iterem totes les dades.
for index, row in df.iterrows():
    # Obtenim la matriu iterada.
    matriu_functional = filtered_normalized_harmonized_functional_matrices[index]
    matriu_structural = filtered_normalized_harmonized_structural_matrices[index]
    matriu_morphological = filtered_normalized_harmonized_morphological_matrices[index]
                        
    # Creem el graf.
    graph_functional = nx.Graph(matriu_iterada)
    graph_structural = nx.Graph(matriu_iterada)
    graph_morphological = nx.Graph(matriu_iterada)
    
    # Obtenim totes les metriques basades en la teoria de grafs.
    result_functional = get_graph_metrics(graph_functional)
    result_structural = get_graph_metrics(graph_structural)
    result_morphological = get_graph_metrics(graph_morphological)
    
    # Informem les metriques al dataframe (functional).
    df.at[index, 'num_edges_f'] = result_functional['num_edges']
    df.at[index, 'mean_degree_f'] = result_functional['mean_degree']
    df.at[index, 'max_degree_f'] = result_functional['max_degree']
    df.at[index, 'mean_density_f'] = result_functional['mean_density']
    df.at[index, 'mean_clustering_coefficient_f'] = result_functional['mean_clust_coef']
    df.at[index, 'assortativity_f'] = result_functional['assortativity']
    df.at[index, 'betweenness_centrality_f'] = result_functional['betw_centr']
    df.at[index, 'global_efficiency_f'] = result_functional['Eglobal']
    df.at[index, 'local_efficiency_f'] = result_functional['Elocal']
    df.at[index, 'transitivity_f'] = result_functional['transitivity']
    df.at[index, 'modularity_f'] = result_functional['modularity']
    
    # Informem les metriques al dataframe (structural).
    df.at[index, 'num_edges_s'] = result_structural['num_edges']
    df.at[index, 'mean_degree_s'] = result_structural['mean_degree']
    df.at[index, 'max_degree_s'] = result_structural['max_degree']
    df.at[index, 'mean_density_s'] = result_structural['mean_density']
    df.at[index, 'mean_clustering_coefficient_s'] = result_structural['mean_clust_coef']
    df.at[index, 'assortativity_s'] = result_structural['assortativity']
    df.at[index, 'betweenness_centrality_s'] = result_structural['betw_centr']
    df.at[index, 'global_efficiency_s'] = result_structural['Eglobal']
    df.at[index, 'local_efficiency_s'] = result_structural['Elocal']
    df.at[index, 'transitivity_s'] = result_structural['transitivity']
    df.at[index, 'modularity_s'] = result_structural['modularity']    

    # Informem les metriques al dataframe (morphological).
    df.at[index, 'num_edges_m'] = result_morphological['num_edges']
    df.at[index, 'mean_degree_m'] = result_morphological['mean_degree']
    df.at[index, 'max_degree_m'] = result_morphological['max_degree']
    df.at[index, 'mean_density_m'] = result_morphological['mean_density']
    df.at[index, 'mean_clustering_coefficient_m'] = result_morphological['mean_clust_coef']
    df.at[index, 'assortativity_m'] = result_morphological['assortativity']
    df.at[index, 'betweenness_centrality_m'] = result_morphological['betw_centr']
    df.at[index, 'global_efficiency_m'] = result_morphological['Eglobal']
    df.at[index, 'local_efficiency_m'] = result_morphological['Elocal']
    df.at[index, 'transitivity_m'] = result_morphological['transitivity']
    df.at[index, 'modularity_m'] = result_morphological['modularity'] 
    

In [104]:
# Export dataframe to CSV.
df.to_csv('output_file_with_metrics.csv', index=False)


In [108]:
df[['num_edges_f', 'mean_degree_f', 'max_degree_f', 'mean_density_f']]

,num_edges_f,mean_degree_f,max_degree_f,mean_density_f
0,1606,42.2632,64,0.563509
1,1606,42.2632,64,0.563509
2,1606,42.2632,64,0.563509
3,1606,42.2632,64,0.563509
4,1606,42.2632,64,0.563509
...,...,...,...,...
160,1606,42.2632,64,0.563509
161,1606,42.2632,64,0.563509
162,1606,42.2632,64,0.563509
163,1606,42.2632,64,0.563509
